In [1]:
import tensorflow as tf
from tensorflow import keras
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

def load_data(resize=(256, 256), batch_size=50):
    import os 
    def __load_data(path):
        sub_paths = os.listdir(path)
        datas = []
        labels = []
        for label, label_name in enumerate(sub_paths):
            for simple in os.listdir(path + os.sep + label_name):
                if simple.endswith('jpg'):
                    labels.append(label)
                    datas.append(path + os.sep + label_name + os.sep + simple)
        datas = tf.data.Dataset.from_tensor_slices(datas)
        labels = tf.data.Dataset.from_tensor_slices(labels)
        def map_img(img_path):
            return tf.image.resize(tf.image.decode_jpeg(tf.io.read_file(img_path)), resize) / 255.
        datas = datas.map(map_img, num_parallel_calls=tf.data.AUTOTUNE)
        ds = tf.data.Dataset.zip((datas, labels))
        # return ds
        return ds.shuffle(buffer_size=len(datas)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        # return datas.batch(batch_size).prefetch(tf.data.AUTOTUNE), labels.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return __load_data('./training_set/'), __load_data('./test_set/')

In [2]:
# 加载数据
BATCH_SIZE = 50
train_iter, test_iter = load_data(batch_size=BATCH_SIZE, resize=(256, 256))

In [3]:
class Inception(tf.keras.Model):
    
    def __init__(self, c1, c2, c3, c4):
        super(Inception, self).__init__()
        self.p1_1 = tf.keras.layers.Conv2D(c1, 1, activation='relu')
        self.p2_1 = tf.keras.layers.Conv2D(c2[0], 1, activation='relu')
        self.p2_2 = tf.keras.layers.Conv2D(c2[1], 3, activation='relu', padding='same')
        self.p3_1 = tf.keras.layers.Conv2D(c3[0], 1, activation='relu')
        self.p3_2 = tf.keras.layers.Conv2D(c3[1], 5, activation='relu', padding='same')
        self.p4_1 = tf.keras.layers.MaxPool2D(3, 1, padding='same')
        self.p4_2 = tf.keras.layers.Conv2D(c4, 1, activation='relu')
    
    def call(self, x):
        p1 = self.p1_1(x)
        p2 = self.p2_2(self.p2_1(x))
        p3 = self.p3_2(self.p3_1(x))
        p4 = self.p4_2(self.p4_1(x))
        return tf.keras.layers.Concatenate()([p1, p2, p3, p4])

In [4]:
def b1():
    return tf.keras.Sequential([
        # filter=64, kernel_size=7  strides=2
        tf.keras.layers.Conv2D(64, 7, strides=2, padding='same', activation='relu'),
        # kernel_size=3, strides=2
        tf.keras.layers.MaxPool2D(3, 2, padding='same')
    ])

def b2():
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, 1, activation='relu'),
        tf.keras.layers.Conv2D(192, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(3, 2, padding='same')
    ])

def b3():
    return tf.keras.Sequential([
        Inception(64, (96, 128), (16, 32), 32),
        Inception(128, (128, 192), (32, 96), 64),
        tf.keras.layers.MaxPool2D(3, 2, padding='same')
    ])

def b4():
    return tf.keras.Sequential([
        Inception(192, (96, 208), (16, 48), 64),
        Inception(160, (112, 224), (24, 64), 64),
        Inception(128, (128, 256), (24, 64), 64),
        Inception(112, (144, 288), (32, 64), 64),
        Inception(256, (160, 320), (32, 128), 128),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')
    ])

def b5():
    return tf.keras.Sequential([
        Inception(256, (160, 320), (32, 128), 128),
        Inception(384, (192, 384), (48, 128), 128),
        tf.keras.layers.GlobalAvgPool2D(),
        tf.keras.layers.Flatten()
    ])

In [5]:
net = tf.keras.Sequential([tf.keras.layers.Input(shape=(256, 256, 3)), b1(), b2(), b3(), b4(), b5(), tf.keras.layers.Dense(2, activation='softmax')])

In [6]:
net.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-3), metrics=['accuracy'])

In [7]:
net.fit(train_iter, epochs=10)

Epoch 1/10


2022-03-26 15:53:49.387317: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.73GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-03-26 15:53:49.387354: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.73GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


161/161 [==============================] - 47s 220ms/step - loss: 0.7732 - accuracy: 0.5033
Epoch 2/10
161/161 [==============================] - 35s 208ms/step - loss: 0.7443 - accuracy: 0.4907
Epoch 3/10
161/161 [==============================] - 35s 208ms/step - loss: 0.7103 - accuracy: 0.4989
Epoch 4/10
161/161 [==============================] - 35s 208ms/step - loss: 0.7290 - accuracy: 0.5252
Epoch 5/10
161/161 [==============================] - 35s 207ms/step - loss: 0.6933 - accuracy: 0.5285
Epoch 6/10
161/161 [==============================] - 35s 207ms/step - loss: 0.7072 - accuracy: 0.5365
Epoch 7/10
161/161 [==============================] - 35s 207ms/step - loss: 0.6882 - accuracy: 0.5417
Epoch 8/10
161/161 [==============================] - 35s 207ms/step - loss: 0.6863 - accuracy: 0.5625
Epoch 9/10
161/161 [==============================] - 35s 207ms/step - loss: 0.6757 - accuracy: 0.5796
Epoch 10/10
161/161 [==============================] - 35s 207ms/step - loss: 0.6737